## Agent with Function Calling and Memory

### What is Function Calling?

Function calling allows AI models to interact with external tools and APIs by generating structured JSON outputs that represent function calls. This enables the model to perform actions beyond text generation, such as retrieving information, performing calculations, or controlling external systems.

- Basic Structure

```json
{
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "simple_function",
        "description": "A simple function that returns a string",
        "parameters": {
          "type": "object",
          "properties": {
            "input_string": {
              "type": "string",
              "description": "A string to pass to the function"
            }
          },
          "required": ["input_string"]
        }
      }
    }
  ]
}
```

#### Key Components Explained

1. **Tools Array**: Contains all available tools/functions the model can call
   - Each tool is defined as an object in this array

2. **Type**: Specifies the kind of tool (currently "function" is the standard type)

3. **Function Object**: Contains the function's metadata
   - **name**: The identifier used to call the function
   - **description**: Explains what the function does (helps the model decide when to use it)
   - **parameters**: Defines the expected input format using JSON Schema

4. **Parameters Object**: Uses JSON Schema to define expected inputs
   - **type**: Usually "object" for structured parameters
   - **properties**: Defines each individual parameter
   - **required**: Lists which parameters must be provided

5. **Individual Parameters**: Each defined with:
   - **type**: Data type (string, number, boolean, array, object)
   - **description**: Explains the parameter's purpose

#### How Function Calling Works

1. The model receives a prompt from the user
2. Based on the prompt, the model determines if a function call is needed
3. If needed, the model generates a structured function call with appropriate parameters
4. The application intercepts this function call and executes the actual function
5. The function's result is returned to the model
6. The model incorporates the result into its response to the user

#### Example Usage Flow

User: "What's the weather in Paris?"

Model (internal reasoning): *This requires current weather data I don't have. I should use a weather function.*

Model (generates function call):
```json
{
  "name": "get_weather",
  "arguments": "{\"location\": \"Paris\", \"unit\": \"celsius\"}"
}
```

System executes `get_weather("Paris", "celsius")` and returns: `{"temperature": 22, "condition": "Sunny"}`

Model (final response): "The weather in Paris is currently sunny with a temperature of 22°C."


### Import necessary libraries

In [24]:
import dotenv
import json
from openai import OpenAI
import pandas as pd
from typing import List, Dict, Literal, Callable

dotenv.load_dotenv()

client = OpenAI()

### Define Functions

First, define the Python functions that will read and write from the project_management.csv file using Pandas dataframes. This code uses Pandas dataframes to read and write from the .csv file. 

We define 3 tasks our project management assistant can perform. 

Each function returns a JSON string as output

In [3]:
# Load the project management data
df = pd.read_csv('project_management.csv')
df

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,John Smith,In Progress,High,2023-08-15,2023-07-01,2023-07-10,3,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,Completed,Medium,2023-08-15,2023-07-01,2023-08-13,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Alice Johnson,Completed,Low,2023-07-15,2023-06-01,2023-07-05,2,2.0,Compile the weekly project status report,Reporting,NaN


In [4]:
def task_retrieval_and_status_updates(task_id, status, last_updated):
    """Retrieve and update task status"""
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.loc[df['Task ID'] == task_id, 'Last Updated'] = last_updated
    df.to_csv('project_management.csv', index=False)  # save changes to file
    task = df.loc[df['Task ID'] == task_id]
    return json.dumps(task.to_dict())

def project_reporting_and_analytics(project_id):
    """Generate reports on project progress and team performance"""
    project = df.loc[df['Project ID'] == project_id]
    return json.dumps(project.to_dict())

def resource_allocation_and_scheduling(task_id, assigned_to, time_estimate, due_date, status):
    """Allocate tasks based on current workloads and schedules"""
    df.loc[df['Task ID'] == task_id, 'Assigned To'] = assigned_to
    df.loc[df['Task ID'] == task_id, 'Time Estimate'] = time_estimate
    df.loc[df['Task ID'] == task_id, 'Due Date'] = due_date
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.to_csv('project_management.csv', index=False)  # save changes to file
    task = df.loc[df['Task ID'] == task_id]
    return json.dumps(task.to_dict())

### Create function calling in hard coded way

Next, we'll build the project management assistant conversation. 

We'll define the messages to send to the model, including a tools dictionary that defines a list of tools, which are the functions that are available to the model to identify and parse parameters for. 

In [5]:
def run_conversation(messages):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "task_retrieval_and_status_updates",
                "description": "Retrieve and update task status",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {
                            "type": "integer",
                            "description": "The unique identifier for the task"
                        },
                        "status": {
                            "type": "string",
                            "description": "The new status of the task"
                        },
                        "last_updated": {
                            "type": "string",
                            "description": "The date of the last status update or change to the task"
                        }
                    },
                    "required": ["task_id", "status", "last_updated"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "project_reporting_and_analytics",
                "description": "Generate reports on project progress and team performance",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "project_id": {
                            "type": "integer",
                            "decription": "The unique identifier for the project"
                        }
                    },
                    "required": ["project_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "resource_allocation_and_scheduling",
                "description": "Allocate tasks based on current workloads and schedules",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {
                            "type": "integer",
                            "description": "The unique identifier for the task"
                        },
                        "assigned_to": {
                            "type": "string",
                            "description": "The user be assigned to certain task"
                        },
                        "time_estimate": {
                            "type": "integer",
                            "description": "Finish the task will consume how many days"
                        },
                        "due_date": {
                            "type": "string",
                            "format": "date-time",
                            "description": "The last day to complete the task"
                        },
                        "status": {
                            "type": "string",
                            "description": "The task status, such as 'Completed' or 'In Progress'"
                        }
                    },
                    "required": ["task_id", "assigned_to", "time_estimate", "due_date", "status"]
                }
            }
        }
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "task_retrieval_and_status_updates": task_retrieval_and_status_updates,
            "project_reporting_and_analytics": project_reporting_and_analytics,
            "resource_allocation_and_scheduling": resource_allocation_and_scheduling,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name == 'task_retrieval_and_status_updates':
                function_response = function_to_call(
                    task_id=function_args.get("task_id"),
                    status=function_args.get("status"),
                    last_updated=function_args.get("last_updated")
                )
            elif function_name == 'project_reporting_and_analytics':
                function_response = function_to_call(
                    project_id=function_args.get("project_id")
                )
            elif function_name == 'resource_allocation_and_scheduling':
                function_response = function_to_call(
                    task_id=function_args.get("task_id"),
                    assigned_to=function_args.get("assigned_to"),
                    time_estimate=function_args.get("time_estimate"),
                    due_date=function_args.get("due_date"),
                    status=function_args.get("status")
                )

            message_to_append = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            messages.append(message_to_append)

        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )
        assistant_response = second_response.choices[0].message.content
        messages.append({
            "role": "assistant",
            "content": assistant_response
        })
        return messages

### Interactive function

In [10]:
messages = [
    {
        "role": "system", 
        "content": "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. You can provide updates on projects, assign tasks to team members, and schedule meetings. You understand project management terminology and are capable of parsing detailed project data. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    }
]

In [11]:
def interactive_conversation(user_prompt, messages):
    messages.append({
        "role": "user",
        "content": user_prompt
    })
    messages = run_conversation(messages)
    assistant_response = messages[-1]["content"]
    print("Assistant:", assistant_response)
    return messages

- Ask for change the task status, this should call function `task_retrieval_and_status_updates`

In [12]:
messages = interactive_conversation("Change the status of task 2 to completed at 2023-08-15", messages)


Assistant: Task 2 "Implement Login Page" has been marked as completed as of August 15, 2023.


In [13]:
df

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,John Smith,In Progress,High,2023-08-15,2023-07-01,2023-07-10,3,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,Completed,Medium,2023-08-15,2023-07-01,2023-08-15,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Alice Johnson,Completed,Low,2023-07-15,2023-06-01,2023-07-05,2,2.0,Compile the weekly project status report,Reporting,NaN


- Ask for query the project report, this should call function `project_reporting_and_analytics`

In [14]:
messages = interactive_conversation("Give me the report of 101 project.", messages)

Assistant: The report for project 101 is as follows:

| Task ID | Task Name              | Assigned To | Status      | Priority | Due Date   | Last Updated | Time Estimate | Time Spent | Project Phase |
|---------|------------------------|-------------|-------------|----------|------------|--------------|----------------|-------------|----------------|
| 1       | Design Database Schema | John Smith  | In Progress | High     | 2023-08-15 | 2023-07-10   | 3              | 4           | Design         |
| 2       | Implement Login Page   | John Smith  | Completed   | Medium   | 2023-08-15 | 2023-08-15   | 5              | NaN         | Implementation |



- Ask for allocate resource and scheduling, this should call function `resource_allocation_and_scheduling`

In [15]:
messages = interactive_conversation("Allocate task 1 of project 101 to John Smith, it will take 3 days to complete, the due date is 2023-08-15", messages)

Assistant: Task 1 "Design Database Schema" of project 101 has been allocated to John Smith with a time estimate of 3 days, and the status has been updated to "In Progress" with a due date of August 15, 2023.


In [16]:
updated_df = pd.read_csv("project_management.csv")
updated_df

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,John Smith,In Progress,High,2023-08-15,2023-07-01,2023-07-10,3,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,Completed,Medium,2023-08-15,2023-07-01,2023-08-15,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Alice Johnson,Completed,Low,2023-07-15,2023-06-01,2023-07-05,2,2.0,Compile the weekly project status report,Reporting,NaN


In [17]:
messages

[{'role': 'system',
  'content': "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. You can provide updates on projects, assign tasks to team members, and schedule meetings. You understand project management terminology and are capable of parsing detailed project data. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': 'Change the status of task 2 to completed at 2023-08-15'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jNqxCgHxZqTpsDD5KyZMTakV', function=Function(arguments='{"task_id":2,"status":"Completed","last_updated":"2023-08-15"}', name='task_retrieval_and_status_updates'), type='function')]),
 {'tool_call_id': 'call_jNqxCgHxZqTpsDD5KyZMTakV',
  'role': 'tool',
  'name': 'task_retrieval_and_status_u

### Adding tool calls into `Memory` class

As you noticed, the previous code we use explictly list to manage the conversation history. This is not scalable and not efficient. We need to use a memory layer (introduced by [01-manage-memory-layer.ipynb](./01-manage-memory-layer.ipynb)) to manage the conversation history.

In order to manage tool calls, we need to further modify the previous Memory class to support tool calls.

In [96]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], # Added 'tool' as a new role type
                    content: str,
                    tool_calls: List = None,    # New parameter to store tool call information
                    tool_call_id = None)-> None:  # New parameter to track specific tool call IDs

        # For regular messages (user/system/assistant), include tool_calls dictionary
        message = {"role": role, "content": content}
        
        # Only add tool_calls if they exist and role is assistant
        if tool_calls and role == "assistant":
            message["tool_calls"] = tool_calls
            
        # Add tool_call_id if it exists and role is tool
        if tool_call_id and role == "tool":
            message["tool_call_id"] = tool_call_id
            
        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

In [97]:
memory = Memory()
memory.add_message(role="system", content="You're a helpful assitant")

In [98]:
memory.get_messages()

[{'role': 'system', 'content': "You're a helpful assitant"}]

### Adding tool calls into `Agent` class

In [100]:
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall
from openai.types.chat.chat_completion_message import ChatCompletionMessage

In [101]:
class Agent:
    def __init__(
        self,
        name: str = "Agent",
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List = [],
        tool_mapping: Dict[str, Callable] = {},  # Mapping function name to function
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.agent = OpenAI()

        self.memory = Memory()
        self.memory.add_message(
            role="system", content=instructions
        )  # Update: memory as class

        self.tool_mapping = tool_mapping
        self.tools = tools

    def _get_completion(
        self, messages: List[Dict], tools: List = None
    ) -> ChatCompletionMessage:
        
        try:
            response = self.agent.chat.completions.create(
                model=self.model,
                temperature=self.temperature,
                messages=messages,
                tools=tools,
            )
        except Exception as e:
            print(f"Error: {e}")

        return response.choices[0].message

    def invoke(self, message: str) -> str:
        self.memory.add_message(role="user", content=message)  # track user input
        response = self._get_completion(
            messages=self.memory.get_messages(),
            tools=self.tools,
        )

        ai_message = str(response.content)
        tool_calls = response.tool_calls  # If the model decides to call a function

        self.memory.add_message(
            role="assistant", content=ai_message, tool_calls=tool_calls
        )  # track ai response

        if tool_calls:
            self._call_tools(tool_calls)  # handle tool calls
            response = self._get_completion(
                messages=self.memory.get_messages(),
            )  # after tool calls, get final response
            ai_message = str(response.content)
            self.memory.add_message(role="assistant", content=ai_message)

        return ai_message

    def _call_tools(self, tool_calls: List[ChatCompletionMessageToolCall]) -> None:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            callable_tool = self.tool_mapping[function_name]
            result = callable_tool(**args)
            self.memory.add_message(
                role="tool", content=str(result), tool_call_id=tool_call.id
            )  # track tool response

In [102]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "task_retrieval_and_status_updates",
            "description": "Retrieve and update task status",
            "parameters": {
                "type": "object",
                "properties": {
                    "task_id": {
                        "type": "integer",
                        "description": "The unique identifier for the task"
                    },
                    "status": {
                        "type": "string",
                        "description": "The new status of the task"
                    },
                    "last_updated": {
                        "type": "string",
                        "description": "The date of the last status update or change to the task"
                    }
                },
                "required": ["task_id", "status", "last_updated"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "project_reporting_and_analytics",
            "description": "Generate reports on project progress and team performance",
            "parameters": {
                "type": "object",
                "properties": {
                    "project_id": {
                        "type": "integer",
                        "decription": "The unique identifier for the project"
                    }
                },
                "required": ["project_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "resource_allocation_and_scheduling",
            "description": "Allocate tasks based on current workloads and schedules",
            "parameters": {
                "type": "object",
                "properties": {
                    "task_id": {
                        "type": "integer",
                        "description": "The unique identifier for the task"
                    },
                    "assigned_to": {
                        "type": "string",
                        "description": "The user be assigned to certain task"
                    },
                    "time_estimate": {
                        "type": "integer",
                        "description": "Finish the task will consume how many days"
                    },
                    "due_date": {
                        "type": "string",
                        "format": "date-time",
                        "description": "The last day to complete the task"
                    },
                    "status": {
                        "type": "string",
                        "description": "The task status, such as 'Completed' or 'In Progress'"
                    }
                },
                "required": ["task_id", "assigned_to", "time_estimate", "due_date", "status"]
            }
        }
    }
]

tool_mapping = {
    "task_retrieval_and_status_updates": task_retrieval_and_status_updates,
    "project_reporting_and_analytics": project_reporting_and_analytics,
    "resource_allocation_and_scheduling": resource_allocation_and_scheduling,
}

system_prompt = "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. You can provide updates on projects, assign tasks to team members, and schedule meetings. You understand project management terminology and are capable of parsing detailed project data. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."


In [103]:
agent = Agent(
    role="Project Management Assistant",
    instructions=system_prompt,
    tools=tools,
    tool_mapping=tool_mapping
)

In [104]:
ai_response = agent.invoke(
    "Change the status of task 2 to completed at 2023-08-13"
)
print(ai_response)

The status of Task 2, "Implement Login Page," has been successfully changed to **Completed** as of **2023-08-13**. 

If you need any further assistance or updates, feel free to ask!


In [105]:
df

,Task ID,Task Name,Project ID,Assigned To,Status,Priority,Due Date,Date Created,Last Updated,Time Estimate,Time Spent,Description,Project Phase,Dependencies
0,1,Design Database Schema,101,John Smith,In Progress,High,2023-08-15,2023-07-01,2023-07-10,3,4.0,Create initial database schema for customer data,Design,NaN
1,2,Implement Login Page,101,John Smith,Completed,Medium,2023-08-15,2023-07-01,2023-08-13,5,NaN,"""Develop the login page UI and backend""",Implementation,1.0
2,3,Prepare Project Report,102,Alice Johnson,Completed,Low,2023-07-15,2023-06-01,2023-07-05,2,2.0,Compile the weekly project status report,Reporting,NaN


In [106]:
agent.memory.get_messages()

[{'role': 'system',
  'content': "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. You can provide updates on projects, assign tasks to team members, and schedule meetings. You understand project management terminology and are capable of parsing detailed project data. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': 'Change the status of task 2 to completed at 2023-08-13'},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_5unDM9IiaOkfJNLhEvYF0gEx', function=Function(arguments='{"task_id":2,"status":"Completed","last_updated":"2023-08-13"}', name='task_retrieval_and_status_updates'), type='function')]},
 {'role': 'tool',
  'content': '{"Task ID": {"1": 2}, "Task Name": {"1": "Implement Login Page"}, "Project ID": {"1": 101}, "Assigned To": {"1": "John Smith"}, "Status": {"1":

In [107]:
ai_response = agent.invoke("Give me the report of 101 project.")
print(ai_response)

Here is the report for Project 101:

### Project Overview
- **Project ID:** 101

### Tasks Overview

| Task ID | Task Name                  | Assigned To  | Status       | Priority | Due Date    | Date Created | Last Updated | Time Estimate (hrs) | Time Spent (hrs) | Description                                      | Project Phase  | Dependencies |
|---------|----------------------------|--------------|--------------|----------|-------------|--------------|--------------|---------------------|------------------|--------------------------------------------------|----------------|--------------|
| 1       | Design Database Schema     | John Smith   | In Progress  | High     | 2023-08-15  | 2023-07-01   | 2023-07-10   | 3                   | 4.0              | Create initial database schema for customer data | Design         | N/A          |
| 2       | Implement Login Page       | John Smith   | Completed    | Medium   | 2023-08-15  | 2023-07-01   | 2023-08-13   | 5                   | N

In [109]:
agent.memory.get_messages()

[{'role': 'system',
  'content': "You are a project management assistant with knowledge of project statuses, task assignments, and scheduling. You can provide updates on projects, assign tasks to team members, and schedule meetings. You understand project management terminology and are capable of parsing detailed project data. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': 'Change the status of task 2 to completed at 2023-08-13'},
 {'role': 'assistant',
  'content': 'None',
  'tool_calls': [ChatCompletionMessageToolCall(id='call_5unDM9IiaOkfJNLhEvYF0gEx', function=Function(arguments='{"task_id":2,"status":"Completed","last_updated":"2023-08-13"}', name='task_retrieval_and_status_updates'), type='function')]},
 {'role': 'tool',
  'content': '{"Task ID": {"1": 2}, "Task Name": {"1": "Implement Login Page"}, "Project ID": {"1": 101}, "Assigned To": {"1": "John Smith"}, "Status": {"1":

### Summary

#### Best Practices

1. Provide clear function descriptions to help the model understand when to use them
2. Use specific parameter descriptions to guide proper parameter usage
3. Mark parameters as required only when absolutely necessary
4. Design functions to be atomic and focused on specific tasks
5. Handle errors gracefully when the model provides invalid parameters

#### Implementation Considerations

- Function calling requires a runtime that can parse the model's JSON output
- The application must implement the actual functions being called
- Consider rate limits and API costs for external service calls
- Add validation to ensure security when executing function calls
